In [1]:

import numpy as np
import functools #feel like we are doing to be caching
from collections import deque
import aocd
from z3 import *

In [2]:
test_data = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""

In [3]:
data = aocd.get_data()

In [4]:
def str_to_bits(string):
    string = "".join(["0" if x == "." else "1" for x in string[1:-1]])
    n = int(string, 2)
    return n, len(string)

In [5]:
x = str_to_bits("[.##.]")

In [6]:
button = 1<<3 | 1<<1

In [7]:
button

10

In [8]:
bin(button)

'0b1010'

In [9]:
bin(6 | 8)

'0b1110'

In [10]:
x = 0
x | (3<<1)

6

In [11]:
# I need to convert a button tuple to bits that can be used in an XOR operation that is a button press. 
# There is some left-right stuff between bitshifting vs the representation of the machines indicator lights.
def convert_button(button, n_bits):
    mask = 0
    for b in button:
        mask |= 1 << (n_bits - 1 - b)  # left index → right index
    return mask

In [12]:
def parse_data(data):
    machines = []
    for line in data.split('\n'):
        linesplit = line.split(" ")
        final_state = linesplit[0]
        final_state, n_bits = str_to_bits(final_state)
        joltages = linesplit[-1]
        buttons = linesplit[1:-1]
        buttons = [tuple(map(int, button.strip("()").split(","))) for button in buttons]
        buttons = [convert_button(button, n_bits) for button in buttons]
        machines.append({"final_state": final_state, "buttons":buttons, "n_bits":n_bits, "joltages":joltages})
    return machines


In [13]:
machines = parse_data(test_data)

In [14]:

def press_buttons(start, final_state, buttons, n_bits):
    mask = (1 << n_bits) - 1

    queue = deque([(start, 0)])
    visited = {start}

    while queue:
        state, steps = queue.popleft()

        if state == final_state:
            return steps

        for b in buttons:
            new_state = (state ^ b) & mask   # Restrict to bit_width
            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, steps + 1))

    return None

In [15]:
m = machines[0]

In [16]:
for m in machines:
    print(m)
    print(press_buttons(0, m["final_state"], m["buttons"], m["n_bits"]))

{'final_state': 6, 'buttons': [1, 5, 2, 3, 10, 12], 'n_bits': 4, 'joltages': '{3,5,4,7}'}
2
{'final_state': 2, 'buttons': [23, 6, 17, 28, 15], 'n_bits': 5, 'joltages': '{7,5,12,7,2}'}
3
{'final_state': 29, 'buttons': [62, 38, 59, 24], 'n_bits': 6, 'joltages': '{10,11,11,5,10,5}'}
2


In [17]:
machines = parse_data(data)
total = 0
for m in machines:
    total += press_buttons(0, m["final_state"], m["buttons"], m["n_bits"])

In [18]:
total

486

In [19]:
#part two: we need a new push button thing and a new final state and everything for the joltages

In [20]:
def get_joltages(joltages):
    joltages = joltages[1:-1].split(",")
    joltages = tuple([int(j) for j in joltages])
    return joltages, len(joltages)

In [21]:
# def convert_button_joltage_mode(button, n_bits):
#     mask = [0 for _ in range(n_bits)]
#     for b in button:
#         mask[b]+=1
#     return tuple(mask)

def convert_button_joltage_mode(button, n_bits):
    return tuple(1 if i in button else 0 for i in range(n_bits))

In [22]:
convert_button_joltage_mode((4,1), 5)

(0, 1, 0, 0, 1)

In [23]:
n_bits = 5
button = (0,2,3,4)
convert_button_joltage_mode(button, n_bits)

(1, 0, 1, 1, 1)

In [24]:
get_joltages('{10,11,11,5,10,5}')

((10, 11, 11, 5, 10, 5), 6)

In [25]:
def parse_data2(data):
    machines = []
    for line in data.split('\n'):
        linesplit = line.split(" ")
        final_state = linesplit[0]
        # final_state, n_bits = str_to_bits(final_state)
        joltages = linesplit[-1]
        joltages, n_bits = get_joltages(joltages)
        buttons = linesplit[1:-1]
        buttons = [tuple(map(int, button.strip("()").split(","))) for button in buttons]
        buttons = [convert_button_joltage_mode(button, n_bits) for button in buttons]
        machines.append({"final_state": final_state, "buttons":buttons, "n_bits":n_bits, "joltages":tuple(joltages)})
    return machines


In [26]:
test_machines = parse_data2(test_data)

In [27]:
j = test_machines[0]["joltages"][0]

In [28]:
b = test_machines[0]["buttons"][1]

In [29]:
def press_buttons2(start, final_state, buttons):
    queue = deque([(start, 0)])
    visited = {start}

    while queue:
        state, steps = queue.popleft()

        if state == final_state:
            return steps

        for b in buttons:
            new_state = tuple(si + bi for si, bi in zip(state, b))
            # new_state = (state ^ b) & mask   # Restrict to bit_width
            if any(ns > fs for ns, fs in zip(new_state, final_state)):
                continue
            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, steps + 1))

    return None

In [30]:
def press_buttons2(start, final_state, buttons):
    n_bits = len(final_state)
    queue = deque([(start, 0)])
    visited = {start}

    while queue:
        state, steps = queue.popleft()

        if state == final_state:
            return steps

        for b in buttons:
            new_state = tuple(si + bi for si, bi in zip(state, b)) #increment the joltage counters

            # if any joltage counter exceeds the target, this is not a combination we can use
            if any(ns > fs for ns, fs in zip(new_state, final_state)):
                continue

            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, steps + 1))

    return None

In [31]:
total = 0
for i, m in enumerate(test_machines):
    start = tuple([0]*m["n_bits"])
    bpresses = press_buttons2(start, m["joltages"], m["buttons"])
    # print(f"final_state = {m['joltages']}, buttons = {m['buttons']}")
    # print(f"Press {bpresses} times for machine {i}")
    total += bpresses
print(total)
    

33


In [32]:
machines = parse_data2(data)

In [33]:
def solve_joltage_z3(target, buttons):
    n_buttons = len(buttons)
    n_bits = len(target)

    # integer variables for button press counts
    xs = [Int(f"x{i}") for i in range(n_buttons)]

    solver = Optimize()

    # non-negative integers
    for x in xs:
        solver.add(x >= 0)

    # add one equation per counter
    for bit in range(n_bits):
        solver.add(
            Sum(xs[i] * buttons[i][bit] for i in range(n_buttons)) == target[bit]
        )

    # objective: minimize total button presses
    total_presses = Sum(xs)
    solver.minimize(total_presses)

    if solver.check() != sat:
        return None

    model = solver.model()
    presses = [model[x].as_long() for x in xs]
    return presses, sum(presses)

In [37]:
total = 0
for i, m in enumerate(machines):
    start = tuple([0]*m["n_bits"])
    # print(f"final_state = {m['joltages']}, buttons = {m['buttons']}")
    bpresses, npresses = solve_joltage_z3(m["joltages"], m["buttons"])
    
    print(f"Press {bpresses} times for macihne {i}")
    total += npresses
print(total)

Press [20, 16, 5, 13, 8, 14, 11] times for macihne 0
Press [21, 0, 9, 22, 16, 9] times for macihne 1
Press [9, 0, 0, 17, 8, 200] times for macihne 2
Press [6, 9, 0, 5] times for macihne 3
Press [0, 13, 1, 7, 6, 0, 18, 5, 14] times for macihne 4
Press [13, 17, 5, 0] times for macihne 5
Press [1, 9, 16, 143, 6, 12, 8, 0, 1, 14, 6] times for macihne 6
Press [7, 12, 18, 0] times for macihne 7
Press [13, 5, 2, 20, 6, 1, 18] times for macihne 8
Press [4, 0, 28, 20, 11, 1, 15, 9, 21] times for macihne 9
Press [12, 17, 20, 11, 2] times for macihne 10
Press [13, 4, 11, 0, 11, 19] times for macihne 11
Press [2, 16, 21, 15, 25, 11, 11, 6, 0, 13, 4] times for macihne 12
Press [19, 19, 2, 18, 6] times for macihne 13
Press [0, 2, 12, 20, 16, 20] times for macihne 14
Press [0, 16, 5, 0, 12, 38, 1, 9, 2, 10, 14, 28, 8] times for macihne 15
Press [129, 15, 19, 12, 4, 20] times for macihne 16
Press [14, 4, 4, 19, 8, 8, 16, 117] times for macihne 17
Press [10, 13, 11, 22, 2, 8, 1, 2, 8] times for macihne

In [ ]:
total